<a href="https://colab.research.google.com/github/Tetsuro-Ishida/tupwards.com/blob/main/CTMRG_sucess1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy import linalg as LA
import itertools
import copy

In [ ]:
class Ising2D:
    def __init__(self, J, chi):
        self.J = J
        self.chi = chi
        self.count = 0
    
    def W(self, a, b, c, d):
        return np.exp(self.J*((2*a-1)*(2*b-1) + (2*b-1)*(2*c-1) + (2*c-1)*(2*d-1) + (2*d-1)*(2*a-1)))
  
    def C(self, c, d):
        C = 0
        for i in np.arange(0, 2):
            for j in np.arange(0, 2):
                C += self.W(i, j, c, d)
        return C

    def P1(self, a, c, d):
        P1 = 0
        for i in np.arange(0, 2):
            P1 += self.W(a, i, c, d)
        return P1

    def P2(self, b, c, d):
        P2 = 0
        for i in np.arange(0, 2):
            P2 += self.W(i, b, c, d)
        return P2

    def C_base(self, a, b, c, d): ## C(2)を作る
        C_base = 0
        for i in np.arange(0, 2):
            for j in np.arange(0, 2):
                for k in np.arange(0, 2):
                    for l in np.arange(0, 2):
                        C_base += self.C(i, j) * self.P1(a, k, i) * self.P2(j, l, d) * self.W(l, k, b, c)
        return C_base

    def set_list(self, L): ## 初めの2の4乗個の項しか使ってない
        num = list(itertools.product([0,1], repeat=L))
        for i in range(len(num)):
            num[i] = list(num[i])
            num[i].reverse()
        num = np.array(num)
        return num
    
    def set_list_non_reversed(self, L): ## non-reversed
        num = list(itertools.product([0,1], repeat=L))
        for i in range(len(num)):
            num[i] = list(num[i])
        num = np.array(num)
        return num

    def C_base_2(self, L=2): ## 4x4の行列を作っている、a, b, が縦の４要素
        C_list_L = np.zeros((2**L, 2**L))
        listC = self.set_list(2*L)
        listC = np.reshape(listC, (4, 4, -1))
        count1 = 0
        count2 = 0
        for i in listC:
            for j in i:
                a, b, c, d = j
                C_list_L[count1, count2] = self.C_base(a, b, c, d)
                count1 += 1
            count1 = 0
            count2 += 1
        return C_list_L
  
    def P_base_2(self): ## 2x4の行列を作っている, aが縦の２要素
        P_list = np.zeros((2*1, 2**2))
        listP = self.set_list_non_reversed(3)
        listP = np.reshape(listP, (2, 4, -1))
        count1 = 0
        count2 = 0
        for i in listP:
            for j in i:
                a, b, c = j
                P_list[count1, count2] = self.P1(a, b, c)
                count2 += 1
            count2 = 0
            count1 += 1
        return P_list
  
    def W_base(self): ## Wの4x4の行列を作る
        W_list = np.zeros((2**2, 2**2))
        listW = self.set_list_non_reversed(4)
        listW = np.reshape(listW, (4, 4, -1))
        count1 = 0
        count2 = 0
        for i in listW:
            for j in i:
                a, b, c, d = j
                W_list[count1, count2] = self.W(a, b, c, d)
                count2 += 1
            count2 = 0
            count1 += 1
        return W_list

    def P1_tilda(self, L, P, v):
        #### まず Wの変形から、Pとのテンソル積を取れるようにする ####
        W = copy.copy(self.W_base()) ## Wの行列も得る
        W = np.hsplit(W, 2) ## Wの行列を縦に真ん中で切る (これで番号②(0), ②(1)を別々に扱う)
        count = 0
        for i in W:
            W[count] = i.reshape(-1)
            W[count] = W[count].reshape(2, -1)
            W[count] = W[count].T
            count += 1
        W = np.concatenate(W, axis=0)
        ## Wの行列が①'から④'までが1行目、⑤'から⑧'までが2行目、というふうに変形される

        ##### 次は PとWのテンソル積をとる ######
        P_size = np.shape(P[0])[0]
        ret = np.zeros((P_size, P_size))
        P_list_L = []
        for k in W:
            for m, l in zip(k, P):
                ret += m*l
            P_list_L.append(ret)
            ret = np.zeros((P_size, P_size))

        P_list_L = np.concatenate(P_list_L, axis=0)

        ### 順番がおかしいので、並び替える ###
        P_list_L = np.vsplit(P_list_L, 2)
        count = 0
        for i in P_list_L:
            P_list_L[count] = np.concatenate(np.vsplit(i, 2), axis=1)
            P_list_L[count] = P_list_L[count].T
            count += 1
        prep = np.arange(0, 2*P_size).reshape(2, -1)
        prep = prep.T
        prep = prep.reshape(-1)
        count = 0
        for i in P_list_L:
            P_list_L[count] = P_list_L[count][prep, :]
            P_list_L[count] = P_list_L[count][:, prep]
            count += 1
        P_L_act = np.concatenate(P_list_L, axis=1)
        
        #### Aによる圧縮 #####
        P1_v1 = []
        
        #### 内積のように計算できる ####
        P1_v = v.T @ P_L_act
        P1_v = np.hsplit(P1_v, 2)
        for i in P1_v:
            ret = i @ v
            P1_v1.append(ret)
        ###### 行が 左側の足(χ個、もしくは 2**(L-1)個)、列も同じ、二つに分かれてるところが上の足 ########
        
        return P1_v1
    

    def P1_C(self, L, C, P1):
        
        P1_C_pre = []
        for i in P1:
            ret = C @ i
            P1_C_pre.append(ret)

        return P1_C_pre

    def P2_C(self, L, P1_C_pre, P2):
        P2_C_act = []
        for i in P2:
            for j in P1_C_pre:
                ret = i @ j
                P2_C_act.append(ret) ### Wとのテンソル積を取るのが、[0, 0], [0, 1], [1, 0], [1, 1] の順に並ぶ

        return P2_C_act

    def C_base_L(self, L, P2_C_act):
        self.count += 1
        ##### 拡大の操作 ##### 
        C = []
        W = self.W_base().T
        P2_C_size = np.shape(P2_C_act[0])[0]
        ret = np.zeros((P2_C_size, P2_C_size))
        for k in W:
            for m, l in zip(k, P2_C_act):
                ret += m*l
            C.append(ret)
            ret = np.zeros((P2_C_size, P2_C_size))
        C_pre1 = np.concatenate([C[0], C[1]], axis=1)
        C_pre2 = np.concatenate([C[2], C[3]], axis=1)
        C_act = np.concatenate([C_pre1, C_pre2], axis=0)

        ### 順番がおかしいので、並び替える ###
        C_size = np.shape(C_act)[0]
        prep = np.arange(0, C_size).reshape(2, -1)
        prep = prep.T
        prep = prep.reshape(-1)
        C_act = C_act[prep, :]
        C_act = C_act[:, prep]
        
        ########## 対角化 ##########
        C_act, self.v = self.Liniag(C_act, L)
        
        return C_act, self.v
        
    def Liniag(self, C_act, L):
        chi = self.chi
        ####### 対角化 #######
        det_C = LA.det(C_act)
        w, v = LA.eig(C_act)
        w = np.sort(abs(w))[::-1]
        w_index = np.argsort(abs(w))
        self.v = v[:, w_index].real
        if chi >= 2**L:
            self.w = np.diag(w)
            C_act = self.w
        elif chi < 2**L:
            w = copy.copy(w[:chi])
            self.w = np.diag(w)
            self.v = self.v[:, :chi].real
            C_act = self.w

        return C_act, self.v
    

In [ ]:
def C_calc(J, L, chi):
    Ising = Ising2D(J, chi)
    dicC = {}
    dicC[str(2)] = Ising.Liniag(Ising.C_base_2(), 2)[0]
    dicP1 = {}
    P = Ising.P_base_2().reshape(4, 2)
    P = np.vsplit(P, 2)
    dicP1[str(2)] = P
    dicP_C = {}
    dicP2 = {}
    dicv = {}
    dicv[str(2)] = Ising.Liniag(Ising.C_base_2(), 2)[1]

    for i in range(2, L):
        dicP1[str(i+1)] = Ising.P1_tilda(L, dicP1[str(i)], dicv[str(i)])
        dicP_C[str(i+1)] = Ising.P1_C(L, dicC[str(i)], dicP1[str(i+1)])
        dicP2[str(i+1)] = Ising.P2_C(L, dicP_C[str(i+1)], dicP1[str(i+1)])
        dicC[str(i+1)], dicv[str(i+1)] = Ising.C_base_L(L, dicP2[str(i+1)])
        
    return dicC

def Z_calc(L, C_):
    Z_par = 0
    num = np.shape(C_)[0]
    for i in range(0, num):
        Z_par += C_[i, i]**4
    return Z_par


In [ ]:
C = C_calc(1, 2, 4)
print(C)

{'2': array([[9833978.62286969,       0.        ,       0.        ,
              0.        ],
       [      0.        , 9728814.23061614,       0.        ,
              0.        ],
       [      0.        ,       0.        ,  190415.80218615,
              0.        ],
       [      0.        ,       0.        ,       0.        ,
         188379.49898881]])}


In [ ]:
Z = Z_calc(2, C['2'])
print(Z)

1.8310862266261197e+28


In [ ]:
# def for_PL(L):
#     if L == 3:
#         P_L = []
#         a = np.arange(2**(2*L-2)).reshape(-1, 4)
#         a = np.hsplit(a, 4)
#         count = 0
#         for i in a:
#             if count%2 == 0:
#                 P_L.append(i)
#             count += 1
#         count = 0
#         for i in a:
#             if count%2 == 1:
#                 P_L.append(i)
#             count += 1
#         P_L = np.concatenate(P_L, axis=1)
#         P_L = P_L.reshape(-1, 2)
#         P_L = np.vsplit(P_L, 2*4**(L-3))
#         P_L = np.concatenate(P_L, axis=1)
#     else:
#         P_L_pre = np.zeros((2**(2*L-3), 2))
#         P_L = for_PL(L-1)
#         print(P_L)
#         count1 = 0
#         for i in P_L:
#             for j in i:
#                 P_L_pre[count1, 0] = j*4
#                 P_L_pre[count1, 1] = j*4+2
#                 count1 += 1
#             for k in i:
#                 P_L_pre[count1, 0] = k*4+1
#                 P_L_pre[count1, 1] = k*4+3
#                 count1 += 1
#         P_L = P_L_pre.reshape(-1, 2**(L-1))
#     return P_L

# for_PL(5)


In [ ]:
################################################
# def for_PL(self, L):
#     if L == 3:
#         P_L = []
#         a = np.arange(2**(2*L-2)).reshape(-1, 4)
#         a = np.hsplit(a, 4)
#         count = 0
#         for i in a:
#             if count%2 == 0:
#                 P_L.append(i)
#             count += 1
#         count = 0
#         for i in a:
#             if count%2 == 1:
#                 P_L.append(i)
#             count += 1
#         P_L = np.concatenate(P_L, axis=1)
#         P_L = P_L.reshape(-1, 2)
#         P_L = np.vsplit(P_L, 2)
#         P_L = np.concatenate(P_L, axis=1)
#     elif L > 3:
#         P_L_pre = np.zeros((2**(2*L-3), 2), dtype=int)
#         P_L = self.for_PL(L-1)
#         count1 = 0
#         for i in P_L:
#             for j in i:
#                 P_L_pre[count1, 0] = int(j*4)
#                 P_L_pre[count1, 1] = int(j*4+2)
#                 count1 += 1
#             for k in i:
#                 P_L_pre[count1, 0] = int(k*4+1)
#                 P_L_pre[count1, 1] = int(k*4+3)
#                 count1 += 1
#         P_L = P_L_pre.reshape(-1, 2*(L-1))
#     return P_L

################################################